<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Quant_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
data_=pd.read_csv('/content/gdrive/MyDrive/Final_Merged_Data/all_data_combined.csv')
data_

,datetime,atm,open,high,low,close,volume,oi,max_pain_strike,intraday_pcr,...,highest_above_vah,lowest_below_val,gap_type,gap_filled,3_day_vah,5_day_vah,3_day_val,5_day_val,3_day_poc,5_day_poc
0,2018-01-01 09:15:00,25570.00,25634.00,25643.10,25608.00,25643.10,480.0,0.0,25700,0.8287,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 09:20:00,25570.00,25631.50,25675.70,25631.50,25665.30,600.0,0.0,25600,0.7786,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 09:25:00,25570.00,25665.30,25665.30,25655.10,25655.10,240.0,0.0,25700,0.7420,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 09:30:00,25570.00,25647.70,25647.70,25647.70,25647.70,240.0,0.0,25600,0.7683,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 09:35:00,25570.00,25647.70,25647.70,25647.00,25647.00,440.0,0.0,25700,0.7973,...,25675.7,25481.3,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92694,2022-12-30 15:10:00,43501.05,43320.05,43349.70,43302.45,43322.00,2650.0,168225.0,42500,0.6826,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92695,2022-12-30 15:15:00,43501.05,43317.30,43331.95,43290.00,43310.00,4950.0,168625.0,42000,0.6204,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92696,2022-12-30 15:20:00,43501.05,43307.35,43443.95,43293.25,43432.90,6600.0,168750.0,42500,0.5152,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7
92697,2022-12-30 15:25:00,43501.05,43433.60,43443.70,43377.05,43403.40,6100.0,168500.0,42000,0.4360,...,43731.8,43165.0,NaN,no,43417.59,43270.46,43105.38,42893.51,43395.52,43244.7


# Mean Reversion from Outside VAH - VAL

all thresholds VAH

In [6]:
import pandas as pd
data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)

def calculate_reversions(data, threshold_percentage):
    move_out_count = 0
    reversion_count = 0
    currently_outside = False

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']

        if prev_vah is not None:
            threshold = prev_vah * threshold_percentage
            if not currently_outside:
                if close_price > prev_vah + threshold:
                    move_out_count += 1
                    currently_outside = True
            else:
                if close_price <= prev_vah:
                    reversion_count += 1
                    currently_outside = False

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False

    return move_out_count, reversion_count

thresholds = [0.0005,0.00075, 0.001, 0.0015, 0.002]
results = {}
for threshold in thresholds:
    move_out_count, reversion_count = calculate_reversions(data, threshold)
    results[f'{threshold * 100}%'] = {
        'Total move outs': move_out_count,
        'Total reversions': reversion_count
    }

for threshold, result in results.items():
    print(f'Threshold {threshold}:')
    print(f"  Total move outs: {result['Total move outs']}")
    print(f"  Total reversions: {result['Total reversions']}")


Threshold 0.05%:
  Total move outs: 459
  Total reversions: 163
Threshold 0.075%:
  Total move outs: 372
  Total reversions: 90
Threshold 0.1%:
  Total move outs: 318
  Total reversions: 49
Threshold 0.15%:
  Total move outs: 269
  Total reversions: 20
Threshold 0.2%:
  Total move outs: 227
  Total reversions: 6


VAL-*Doubt*

In [12]:
import pandas as pd
data = data_.copy()

data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_val'] = data['val'].shift(1)
def calculate_val_reversions(data, threshold_percentage):
    move_out_count = 0
    reversion_count = 0
    currently_outside = False

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_val = data.loc[i, 'prev_val']

        if prev_val is not None:
            threshold = prev_val * threshold_percentage
            if not currently_outside:
                if close_price < prev_val - threshold:
                    move_out_count += 1
                    currently_outside = True
            else:
                if close_price >= prev_val:
                    reversion_count += 1
                    currently_outside = False

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False

    return move_out_count, reversion_count

threshold = 0.001  # 0.1%
move_out_count_val, reversion_count_val = calculate_val_reversions(data, threshold)

print("VAL Results:")
print(f'Threshold {threshold * 100}%:')
print(f"  Total move outs: {move_out_count_val}")
print(f"  Total reversions: {reversion_count_val}")


VAL Results:
Threshold 0.1%:
  Total move outs: 2600
  Total reversions: 2210


In [7]:
import pandas as pd
from collections import defaultdict

data = data_.copy()
data['datetime'] = pd.to_datetime(data['datetime'])
data['prev_vah'] = data['vah'].shift(1)
data['prev_val'] = data['val'].shift(1)

def track_reversion_levels(data):
    reversion_counts = defaultdict(int)
    currently_outside = False
    max_percentage_move_out = 0

    for i in range(1, len(data)):
        close_price = data.loc[i, 'close']
        prev_vah = data.loc[i, 'prev_vah']

        if prev_vah is not None:
            if not currently_outside:
                if close_price > prev_vah:
                    currently_outside = True
                    max_percentage_move_out = ((close_price - prev_vah) / prev_vah) * 100
            else:
                current_percentage_move_out = ((close_price - prev_vah) / prev_vah) * 100
                if current_percentage_move_out > max_percentage_move_out:
                    max_percentage_move_out = current_percentage_move_out

                if close_price <= prev_vah:
                    reversion_counts[round(max_percentage_move_out, 1)] += 1
                    currently_outside = False

            if data.loc[i, 'datetime'].date() != data.loc[i - 1, 'datetime'].date():
                currently_outside = False

    return reversion_counts

reversion_levels = track_reversion_levels(data)
print("Reversion levels and counts:")
for level, count in sorted(reversion_levels.items()):
    print(f"  Reversion from {level}% above VAH: {count} times")


Reversion levels and counts:
  Reversion from 0.0% above VAH: 616 times
  Reversion from 0.1% above VAH: 143 times
  Reversion from 0.2% above VAH: 16 times
  Reversion from 0.3% above VAH: 2 times
  Reversion from 0.6% above VAH: 1 times
  Reversion from 0.7% above VAH: 1 times


# Higher POC

In [3]:
import pandas as pd
data = data_
data['datetime'] = pd.to_datetime(data['datetime'])
data['date'] = data['datetime'].dt.date
data.sort_values(by='datetime', inplace=True)
daily_data = data.groupby('date').agg({
    'poc': 'first',  # POC is the same for the entire day, taking the first occurrence
}).reset_index()

daily_data['poc_change'] = daily_data['poc'].pct_change()
daily_data['poc_trending_higher'] = (
    (daily_data['poc'] > daily_data['poc'].shift(1) * 1.002) &
    (daily_data['poc'].shift(1) > daily_data['poc'].shift(2) * 1.002)
)

trending_higher_dates = daily_data[daily_data['poc_trending_higher']]['date']
results = []

def get_previous_trading_day(date, all_dates):
    previous_dates = all_dates[all_dates < date]
    if not previous_dates.empty:
        return previous_dates.max()
    return None

unique_dates = pd.Series(data['date'].unique()).sort_values()
near_ranges = [0.01,0.015, 0.02, 0.025]  # 1.5%, 2%, 2.5%

for date in trending_higher_dates:
    prev_date = get_previous_trading_day(date, unique_dates)

    if prev_date:
        prev_day_data = data[data['date'] == prev_date]
        local_min = prev_day_data['close'].min()
        daily_subset = data[data['date'] == date]
        for near_range in near_ranges:
            if ((daily_subset['close'] <= local_min * (1 + near_range)) &
                (daily_subset['close'] >= local_min * (1 - near_range))).any():

                if len(daily_subset) >= 2:
                    last_5min_close = daily_subset.iloc[-1]['close']
                    movement_pct = ((last_5min_close - local_min) / local_min) * 100
                    results.append({'date': date, 'near_range': near_range, 'movement_pct': movement_pct})

results_df = pd.DataFrame(results)
summary_stats = results_df.groupby('near_range')['movement_pct'].describe()
#print(summary_stats)
#print(results_df.head())
total_unique_dates = results_df.groupby('near_range')['date'].nunique()
total_pct_change = results_df.groupby('near_range')['movement_pct'].sum()
bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)
range_counts = results_df.groupby(['near_range', 'movement_range']).size().unstack(fill_value=0)
print("Total Unique Number of Dates:")
print(total_unique_dates)
print("Total Percentage Change:")
print(total_pct_change)
print("Movement Percentage Range Counts:")
print(range_counts)


Total Unique Number of Dates:
near_range
0.010    128
0.015    182
0.020    204
0.025    220
Name: date, dtype: int64
Total Percentage Change:
near_range
0.010    163.568467
0.015    286.174179
0.020    349.437409
0.025    403.065061
Name: movement_pct, dtype: float64
Movement Percentage Range Counts:
movement_range  <-2%  -2%-0%  0%-2%  >2%
near_range                              
0.010              0       6     99   23
0.015              0       6    124   52
0.020              0       6    129   69
0.025              0       6    129   85


In [ ]:
'''# Total unique number of dates
total_unique_dates = len(daily_data)

# Total percentage change
total_pct_change = results_df['movement_pct'].sum()

# Divide the pct_change values into ranges
bins = [-float('inf'), -2, 0, 2, float('inf')]
labels = ['<-2%', '-2%-0%', '0%-2%', '>2%']
results_df['movement_range'] = pd.cut(results_df['movement_pct'], bins=bins, labels=labels)

# Count occurrences in each range
range_counts = results_df['movement_range'].value_counts()

# Display results
print("Total Unique Number of Dates:", total_unique_dates)
print("Total Percentage Change:", total_pct_change)
print("Movement Percentage Range Counts:")
print(range_counts)'''


# Local Max-Minima

In [ ]:
df=data_.copy()
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

df['date'] = df.index.date
local_max_min = df.groupby('date')[['local_max', 'local_min']].first().shift(1)
df = df.merge(local_max_min, left_on='date', right_index=True, suffixes=('', '_prev'))

def check_rebound_fall(row):
    if pd.notna(row['local_max_prev']) and row['close'] >= row['local_max_prev']:
        return 'fall'
    elif pd.notna(row['local_min_prev']) and row['close'] <= row['local_min_prev']:
        return 'rebound'
    return None

df['signal'] = df.apply(check_rebound_fall, axis=1)
def calculate_movement(df):
    last_close = df['close'].iloc[-1]
    df['percentage_movement'] = df.apply(lambda row: ((last_close - row['close']) / row['close']) * 100 if row['signal'] else None, axis=1)
    return df

df = df.groupby('date').apply(calculate_movement)
df.reset_index(drop=True, inplace=True)
result = df[df['signal'].notna()][['date', 'close', 'signal', 'percentage_movement']]
result.head()

In [ ]:
signal_counts = result['signal'].value_counts()
total_movement = result.groupby('signal')['percentage_movement'].sum()
movement_value_counts = result.groupby('signal')['percentage_movement'].apply(lambda x: pd.cut(x, bins=4).value_counts())
signal_counts, total_movement, movement_value_counts

In [23]:
df=data_
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

df['date'] = df.index.date
local_max_min = df.groupby('date')[['local_max', 'local_min']].first().shift(1)
df = df.merge(local_max_min, left_on='date', right_index=True, suffixes=('', '_prev'))

def check_rebound_fall(row):
    if pd.notna(row['local_max_prev']) and row['close'] >= row['local_max_prev']:
        return 'fall'
    elif pd.notna(row['local_min_prev']) and row['close'] <= row['local_min_prev']:
        return 'rebound'
    return None

df['signal'] = df.apply(check_rebound_fall, axis=1)
df = df[df['signal'].notna()]
first_occurrence_per_day = df.groupby('date').first().reset_index()

def calculate_movement(row, last_closes):
    last_close = last_closes[row['date']]
    return ((last_close - row['close']) / row['close']) * 100

last_closes = df.groupby('date')['close'].last().to_dict()
first_occurrence_per_day['percentage_movement'] = first_occurrence_per_day.apply(calculate_movement, last_closes=last_closes, axis=1)

result = first_occurrence_per_day[['date', 'close', 'signal', 'percentage_movement']]
signal_counts = result['signal'].value_counts()
total_movement = result.groupby('signal')['percentage_movement'].sum()
movement_value_counts = result.groupby('signal')['percentage_movement'].apply(lambda x: pd.cut(x, bins=4).value_counts())
signal_counts, total_movement, movement_value_counts

(signal
 fall       512
 rebound    423
 Name: count, dtype: int64,
 signal
 fall       105.533789
 rebound   -100.856700
 Name: percentage_movement, dtype: float64,
 signal                    
 fall     (-3.192, 0.17]       278
          (0.17, 3.532]        221
          (-6.568, -3.192]       7
          (3.532, 6.895]         6
 rebound  (-2.281, 2.004]      401
          (-6.566, -2.281]      14
          (2.004, 6.289]         7
          (-10.867, -6.566]      1
 Name: percentage_movement, dtype: int64)

# VWAP REJECTION

In [13]:
# Assuming you have a DataFrame named 'data' containing your dataset with columns 'close', 'vwap', and 'max_pain'
data=data_
# Define a threshold for closely aligned Max Pain and VWAP
alignment_threshold = 0.02  # For example, within 5%

# Initialize variables to track short opportunities and the current rejection movement
short_opportunities = 0
in_rejection = False

# Loop through each row in the DataFrame starting from the second row
for i in range(1, len(data)):
    close_price_previous = data.iloc[i - 1]['close']
    close_price_current = data.iloc[i]['close']
    vwap_price_current = data.iloc[i]['vwap']
    max_pain_price_current = data.iloc[i]['max_pain_strike']

    # Check if the current close price is approaching VWAP from below and then reversing
    if close_price_previous < vwap_price_current and close_price_current > vwap_price_current:
        # Check if Max Pain and VWAP are closely aligned
        if abs(max_pain_price_current - vwap_price_current) / vwap_price_current <= alignment_threshold:
            if not in_rejection:
                in_rejection = True
                short_opportunities += 1
    else:
        # Reset the rejection movement if the previous candle close is broken
        in_rejection = False

# Print the number of short opportunities identified
print("Number of short opportunities:", short_opportunities)


Number of short opportunities: 257


In [14]:
# Assuming you have a DataFrame named 'data' containing your dataset with columns 'close' and 'vwap'

# Initialize variables to track the total number of approaches to VWAP and the number of rejected instances
approaches_to_vwap = 0
rejected_instances = 0

# Initialize variables to track the current rejection movement
in_rejection = False

# Loop through each row in the DataFrame starting from the second row
for i in range(1, len(data)):
    close_price_previous = data.iloc[i - 1]['close']
    close_price_current = data.iloc[i]['close']
    vwap_price_current = data.iloc[i]['vwap']

    # Check if the current close price is approaching VWAP
    if close_price_previous < vwap_price_current and close_price_current > vwap_price_current:
        approaches_to_vwap += 1

        # Set the rejection threshold as 70% of the rise from the approach to the rejection point
        rejection_threshold = close_price_current - close_price_previous * 0.7

        # Check if the rejection is successful
        for j in range(i, len(data)):
            if data.iloc[j]['close'] <= rejection_threshold:
                rejected_instances += 1
                break

# Print the total number of approaches to VWAP and the number of rejected instances
print("Total approaches to VWAP:", approaches_to_vwap)
print("Number of rejected instances:", rejected_instances)


Total approaches to VWAP: 277
Number of rejected instances: 0


# Higher Profiles

In [10]:
df=data_.copy()
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)

# Ensure the dates are properly handled
df['date'] = df.index.date

# Calculate the percentage increase in VAH
df['prev_vah'] = df.groupby('date')['vah'].shift(1)
df['prev_prev_vah'] = df.groupby('date')['vah'].shift(2)
df['vah_increase'] = ((df['vah'] - df['prev_vah']) / df['prev_vah']) * 100
df['prev_vah_increase'] = ((df['prev_vah'] - df['prev_prev_vah']) / df['prev_prev_vah']) * 100

# Identify days with at least 0.1% increase in VAH for two consecutive days
df['higher_profiles'] = (df['vah_increase'] >= 0.05) & (df['prev_vah_increase'] >= 0.05)

# Filter the days with higher profiles
higher_profile_days = df[df['higher_profiles']].dropna()

# Define the function to identify the first signal of the day
def identify_first_signal(df, higher_profile_days):
    first_signals = []

    for date, day_data in df.groupby('date'):
        if date in higher_profile_days['date'].values:
            prev_day = date - pd.Timedelta(days=1)
            if prev_day in df.index.date:
                prev_val = df.loc[df.index.date == prev_day, 'val'].values[0]
                prev_close = df.loc[df.index.date == prev_day, 'close'].values[-1]
                vah = df.loc[df.index.date == prev_day, 'vah'].values[0]
                val_range = vah - prev_val
                target_price = prev_val + 0.5 * val_range

                # Check if today's open is below or at the previous day's VAL or within the previous day's end-of-day range
                open_price = day_data.iloc[0]['open']
                if open_price <= prev_val or (open_price >= prev_val and open_price <= prev_close):
                    for idx, row in day_data.iterrows():
                        if row['high'] >= target_price:
                            first_signals.append((date, 'long', row.name, row['close'], target_price))
                            break

    return first_signals

# Get the first signals
first_signals = identify_first_signal(df, higher_profile_days)

# Create a DataFrame from the first signals
first_signals_df = pd.DataFrame(first_signals, columns=['date', 'signal', 'signal_time', 'signal_close', 'target_price'])

# Calculate the count of trades that met the criteria
total_trades = len(first_signals_df)
successful_trades = len(first_signals_df[first_signals_df['signal'] == 'long'])

# Display results
print(f"Total trades meeting the criteria: {total_trades}")
print(f"Successful trades meeting the criteria: {successful_trades}")


Total trades meeting the criteria: 0
Successful trades meeting the criteria: 0
